In [ ]:
!pip install streamlit bcrypt PyJWT pyngrok textstat PyPDF2

In [ ]:
%%writefile config.py
import os

JWT_SECRET = os.getenv("JWT_SECRET_KEY")
JWT_ALGORITHM = "HS256"
TOKEN_EXPIRY_MINUTES = 60

MAX_LOGIN_ATTEMPTS = 3
LOCK_TIME_MINUTES = 5
PASSWORD_HISTORY_COUNT = 3

EMAIL_ID = os.getenv("EMAIL_ID")
EMAIL_APP_PASSWORD = os.getenv("EMAIL_APP_PASSWORD")

ADMIN_EMAIL = os.getenv("ADMIN_EMAIL_ID")
ADMIN_PASSWORD = os.getenv("ADMIN_PASSWORD")

Writing config.py


In [ ]:
%%writefile db.py
import sqlite3
import json
from datetime import datetime

DB_NAME = "policynav_users.db"

def get_connection():
    return sqlite3.connect(DB_NAME, check_same_thread=False)

def init_db():
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE,
        email TEXT UNIQUE,
        password_hash TEXT,
        security_question TEXT,
        security_answer_hash TEXT,
        failed_attempts INTEGER DEFAULT 0,
        lock_until TEXT,
        password_history TEXT
    )
    """)

    conn.commit()
    conn.close()

def create_user(username, email, password_hash, question, answer_hash):
    conn = get_connection()
    cursor = conn.cursor()
    try:
        history = json.dumps([password_hash])
        cursor.execute("""
        INSERT INTO users (username, email, password_hash, security_question,
                           security_answer_hash, password_history)
        VALUES (?, ?, ?, ?, ?, ?)
        """, (username, email, password_hash, question, answer_hash, history))
        conn.commit()
        return True, "User created"
    except:
        return False, "User exists"
    finally:
        conn.close()

def get_user_by_email(email):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users WHERE email=?", (email,))
    user = cursor.fetchone()
    conn.close()
    return user

def update_login_attempts(email, attempts, lock_until=None):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute(
        "UPDATE users SET failed_attempts=?, lock_until=? WHERE email=?",
        (attempts, lock_until, email)
    )
    conn.commit()
    conn.close()

def update_password(email, new_hash):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("UPDATE users SET password_hash=? WHERE email=?", (new_hash, email))
    conn.commit()
    conn.close()

def update_password_history(email, history_json):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute(
        "UPDATE users SET password_history=? WHERE email=?",
        (history_json, email)
    )
    conn.commit()
    conn.close()

def get_all_users():
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT username, email, failed_attempts, lock_until FROM users")
    rows = cursor.fetchall()
    conn.close()
    return rows

Writing db.py


In [ ]:
%%writefile auth.py
import re
import bcrypt
import jwt
import random
import smtplib
from datetime import datetime, timedelta
from email.mime.text import MIMEText
from config import *

# ---------- HASHING ----------
def hash_text(text: str) -> str:
    return bcrypt.hashpw(text.encode(), bcrypt.gensalt()).decode()

def verify_text(text: str, hashed: str) -> bool:
    return bcrypt.checkpw(text.encode(), hashed.encode())

# ---------- EMAIL VALIDATION ----------
def validate_email(email: str) -> bool:
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email) is not None

# ---------- PASSWORD VALIDATION ----------
def validate_password(password: str):
    if len(password) < 8:
        return False, "Password must be at least 8 characters"
    if not re.search(r"[A-Z]", password):
        return False, "Password must contain at least 1 uppercase letter"
    if not re.search(r"[a-z]", password):
        return False, "Password must contain at least 1 lowercase letter"
    if not re.search(r"\d", password):
        return False, "Password must contain at least 1 number"
    return True, "Valid password"

# ---------- JWT ----------
def generate_token(email: str):
    payload = {
        "email": email,
        "exp": datetime.utcnow() + timedelta(minutes=TOKEN_EXPIRY_MINUTES)
    }
    return jwt.encode(payload, JWT_SECRET, algorithm=JWT_ALGORITHM)

def verify_token(token: str):
    try:
        return jwt.decode(token, JWT_SECRET, algorithms=[JWT_ALGORITHM])
    except:
        return None

# ---------- OTP ----------
def generate_otp():
    return str(random.randint(100000, 999999))

def send_otp_email(receiver, otp):
    msg = MIMEText(f"Your PolicyNav OTP is: {otp}")
    msg["Subject"] = "PolicyNav OTP Verification"
    msg["From"] = EMAIL_ID
    msg["To"] = receiver

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(EMAIL_ID, EMAIL_APP_PASSWORD)
        server.send_message(msg)

Writing auth.py


In [33]:
%%writefile readability.py
import textstat

class ReadabilityAnalyzer:
    def __init__(self, text):
        self.text = text

    def get_all_metrics(self):
        return {
            "Flesch Reading Ease": textstat.flesch_reading_ease(self.text),
            "Flesch-Kincaid Grade": textstat.flesch_kincaid_grade(self.text),
            "SMOG Index": textstat.smog_index(self.text),
            "Gunning Fog": textstat.gunning_fog(self.text),
            "Coleman-Liau": textstat.coleman_liau_index(self.text)
        }

Overwriting readability.py


In [ ]:
%%writefile app.py
import os
import PyPDF2
from readability import ReadabilityAnalyzer
import streamlit as st
import json
from datetime import datetime, timedelta
from db import *
from auth import *
from config import *
import base64

init_db()

st.set_page_config(page_title="PolicyNav Auth", layout="centered")

# ---------- BACKGROUND ----------
def set_bg(image_file="99.jpg"):
    if os.path.exists(image_file):
        with open(image_file, "rb") as f:
            encoded = base64.b64encode(f.read()).decode()

        bg_style = f"""
        <style>
        .stApp {{
            background: url("data:image/jpg;base64,{encoded}") no-repeat center center fixed;
            background-size: cover;
        }}
        </style>
        """
    else:
        bg_style = """
        <style>
        .stApp {
            background: linear-gradient(135deg, #000000, #1c1c1c);
            color: white;
        }
        </style>
        """

    st.markdown(bg_style, unsafe_allow_html=True)

set_bg()

# ---------- SESSION STATE ----------
if "token" not in st.session_state:
    st.session_state.token = None

if "page" not in st.session_state:
    st.session_state.page = "Login"

if "reset_stage" not in st.session_state:
    st.session_state.reset_stage = 0

# ---------- NAVIGATION ----------
def go_to(page):
    st.session_state.page = page
    st.rerun()

# ---------- LOGIN ----------
def login_page():
    st.title("PolicyNav – Login")

    with st.form("login_form"):

        email = st.text_input("Email")
        password = st.text_input("Password", type="password")

        submitted = st.form_submit_button("Login")

        if submitted:

            if not email:
                st.toast("Email required", icon="⚠️")
                return

            if not password:
                st.toast("Password required", icon="⚠️")
                return

            if email == ADMIN_EMAIL and password == ADMIN_PASSWORD:
                go_to("AdminDashboard")

            user = get_user_by_email(email)
            if not user:
                st.toast("Email not registered", icon="❌")
                return

            failed_attempts = user[6]
            lock_until = user[7]

            if lock_until and datetime.utcnow() < datetime.fromisoformat(lock_until):
                st.toast("Account locked. Try again later.", icon="🔒")
                return

            if not verify_text(password, user[3]):

                failed_attempts += 1

                if failed_attempts >= MAX_LOGIN_ATTEMPTS:
                    lock_time = datetime.utcnow() + timedelta(minutes=LOCK_TIME_MINUTES)
                    update_login_attempts(email, failed_attempts, lock_time.isoformat())
                    st.toast("Account locked for 5 minutes", icon="🔒")
                else:
                    update_login_attempts(email, failed_attempts)
                    st.toast("Invalid credentials", icon="❌")

                return

            update_login_attempts(email, 0, None)

            otp = generate_otp()
            send_otp_email(email, otp)

            st.session_state.pending_email = email
            st.session_state.otp = otp

            go_to("OTP")

    col1, col2 = st.columns(2)

    if col1.button("Signup"):
        go_to("Signup")

    if col2.button("Forgot Password"):
        go_to("Forgot")

# ---------- OTP ----------
def otp_page():
    st.title("OTP Verification")

    entered = st.text_input("Enter OTP")

    if st.button("Verify OTP"):

        if entered == st.session_state.otp:
            st.session_state.token = generate_token(st.session_state.pending_email)
            go_to("Dashboard")
        else:
            st.toast("Invalid OTP", icon="❌")

    if st.button("⬅ Back to Login"):
        go_to("Login")

# ---------- SIGNUP ----------
def signup_page():
    st.title("Signup")

    username = st.text_input("Username")
    email = st.text_input("Email")
    password = st.text_input("Password", type="password")
    confirm = st.text_input("Confirm Password", type="password")

    question = st.selectbox("Security Question", [
        "What is your favorite book?",
        "What is your dream job?"
    ])

    answer = st.text_input("Security Answer")

    if st.button("Create Account"):

        if not validate_email(email):
            st.toast("Invalid email format", icon="⚠️")
            return

        valid, msg = validate_password(password)
        if not valid:
            st.toast(msg, icon="⚠️")
            return

        if password != confirm:
            st.toast("Passwords do not match", icon="⚠️")
            return

        success, msg = create_user(
            username,
            email,
            hash_text(password),
            question,
            hash_text(answer)
        )

        if success:
            st.toast("Account created successfully", icon="✅")
            go_to("Login")
        else:
            st.toast(msg, icon="❌")

    if st.button("⬅ Back to Login"):
        go_to("Login")

# ---------- DASHBOARD ----------
def dashboard_page():
    payload = verify_token(st.session_state.token)

    if not payload:
        st.toast("Session expired", icon="❌")
        go_to("Login")

    user = get_user_by_email(payload["email"])
    st.title(f"Welcome, {user[1]} 👋")

    col1, col2 = st.columns(2)

    if col1.button("Logout"):
        st.session_state.token = None
        go_to("Login")

    if col2.button("Reset Password"):
        go_to("Reset")

    if st.button("Readability Analyzer"):
        go_to("Readability")

# ---------- RESET ----------
def reset_page():
    st.title("🔒 Reset Password")

    payload = verify_token(st.session_state.token)

    if not payload:
        st.toast("Session expired", icon="❌")
        go_to("Login")

    user = get_user_by_email(payload["email"])

    old_password = st.text_input("Old Password", type="password")
    new_password = st.text_input("New Password", type="password")

    if st.button("Update Password"):

        if not verify_text(old_password, user[3]):
            st.toast("Old password incorrect", icon="❌")
            return

        history = json.loads(user[8] or "[]")

        for old_hash in history:
            if verify_text(new_password, old_hash):
                st.toast("Cannot reuse old password", icon="⚠️")
                return

        new_hash = hash_text(new_password)

        history.insert(0, new_hash)
        history = history[:PASSWORD_HISTORY_COUNT]

        update_password(payload["email"], new_hash)
        update_password_history(payload["email"], json.dumps(history))

        st.toast("Password updated", icon="✅")
        st.session_state.token = None
        go_to("Login")

    if st.button("⬅ Back to Dashboard"):
        go_to("Dashboard")

# ---------- READABILITY ----------
def readability_page():
    st.title("📘 Text Readability Analyzer")

    payload = verify_token(st.session_state.token)

    if not payload:
        st.toast("Session expired", icon="❌")
        go_to("Login")

    option = st.radio("Choose Input Type", ["Enter Text", "Upload File"])

    text = ""

    if option == "Enter Text":
        text = st.text_area("Enter text to analyze")

    else:
        uploaded_file = st.file_uploader("Upload TXT or PDF", type=["txt", "pdf"])

        if uploaded_file:

            if uploaded_file.type == "text/plain":
                text = uploaded_file.read().decode("utf-8")

            elif uploaded_file.type == "application/pdf":
                pdf_reader = PyPDF2.PdfReader(uploaded_file)
                for page in pdf_reader.pages:
                    text += page.extract_text() or ""

    if st.button("Analyze Readability"):

        if not text.strip():
            st.toast("No text found to analyze", icon="⚠️")
            return

        analyzer = ReadabilityAnalyzer(text)
        metrics = analyzer.get_all_metrics()

        st.subheader("Results")

        for key, value in metrics.items():
            st.write(f"**{key}:** {value}")

    if st.button("⬅ Back to Dashboard"):
        go_to("Dashboard")

# ---------- ADMIN ----------
def admin_dashboard():
    st.title("Admin Dashboard")
    st.write(get_all_users())

    if st.button("Logout"):
        go_to("Login")

# ---------- ROUTER ----------
page = st.session_state.page

if page == "Signup":
    signup_page()
elif page == "Dashboard":
    dashboard_page()
elif page == "Reset":
    reset_page()
elif page == "Forgot":
    go_to("Login")
elif page == "OTP":
    otp_page()
elif page == "AdminDashboard":
    admin_dashboard()
elif page == "Readability":
    readability_page()
else:
    login_page()

Overwriting app.py


In [ ]:
from google.colab import userdata
import os

os.environ["JWT_SECRET_KEY"] = userdata.get("JWT_SECRET_KEY")
os.environ["EMAIL_ID"] = userdata.get("EMAIL_ID")
os.environ["EMAIL_APP_PASSWORD"] = userdata.get("EMAIL_APP_PASSWORD")
os.environ["ADMIN_EMAIL_ID"] = userdata.get("ADMIN_EMAIL_ID")
os.environ["ADMIN_PASSWORD"] = userdata.get("ADMIN_PASSWORD")

In [35]:
!streamlit run app.py &>/content/log.txt &

In [34]:
!pkill streamlit
!pkill ngrok

In [36]:
from pyngrok import ngrok
from google.colab import userdata
token=userdata.get("NGROK_AUTHTOKEN")
ngrok.set_auth_token(token)
print(ngrok.connect(8501))


NgrokTunnel: "https://2993-34-16-237-176.ngrok-free.app" -> "http://localhost:8501"
